In [33]:
import pandas as pd

df_nirvana = pd.read_csv('Dados/nirvana_lyrics.csv')
df_pink_floyd = pd.read_csv('Dados\pink_floyd_lyrics.csv')

In [34]:
df_nirvana_treino = df_nirvana.iloc[:50]
df_pink_floyd_treino = df_pink_floyd.iloc[:50]

df_nirvana_teste = df_nirvana.iloc[50:100]
df_pink_floyd_teste = df_pink_floyd.iloc[50:100]

In [44]:
probabilidade_nirvana = len(df_nirvana_treino) / (len(df_nirvana_treino) + len(df_pink_floyd_treino))
probabilidade_pink_floyd = len(df_pink_floyd_treino) / (len(df_nirvana_treino) + len(df_pink_floyd_treino))

In [60]:
# df_nirvana_treino['lyrics'] --> LISTA
# df_pink_floyd_treino['lyrics'] --> STRING

def split_function_pf(text):
    return text.split()

def split_function_n(text):
    for versos in text:
        return versos.split()

def replace_function_pf(text):
    caracteres_paia = ['.','!','@','#','$','%','&','*','(',')','{','}',':',';','_','?','/','<','>','+','=','|',',','[',']','\n']
    for caractere in text:
        if caractere in caracteres_paia:
            text.replace(caractere, '')
    return text

def replace_function_n(text):
    caracteres_paia = ['.','!','@','#','$','%','&','*','(',')','{','}',':',';','_','?','/','<','>','+','=','|',',','[',']','\n']
    for palavra in text:
        for caractere in palavra:
            if caractere in caracteres_paia:
                text.replace(caractere, '')
    return text

def lower_pf(text):
    for words in text:
        words = words.lower()
    return text

def lower_n(text):
    for words in text:
        words = words.lower()
    return text

0     "Moon in both [houses]..."...Scorpio, [Arabian...
1     Lucifer Sam, siam cat\nAlways sitting by your ...
2     There was a king who ruled the land\nHis Majes...
3     Alone in the clouds all blue\nLying on an eide...
4     TCH TCH\nAHH (AHH)\nTCH TCH\nAHH AHH\nDoi doi\...
5     Doctor, doctor!\nI’m in bed (Doctor, doctor)\n...
6                                                   NaN
7     I want to tell you a story\nBout’ a little man...
8     All movement is accomplished in six stages\nAn...
9     The black and green scarecrow as everyone know...
10    I’ve got a bike, you can ride it if you like\n...
11    Emily tries but misunderstands (ah ooh)\nShe’s...
12    Far, far, far, far away – way\nPeople heard hi...
13    Remember a day before today\nA day when you we...
14    Little by little the night turns around\nCount...
15    Corporal Clegg had a wooden leg\nHe won it in ...
16                                                  NaN
17    Marigolds are very much in love\nBut he do

C:\Users\gusri\AppData\Local\Temp\ipykernel_18688\1249043079.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pink_floyd_treino['lyrics'] = replace_function_pf(df_pink_floyd_treino['lyrics'])
